# SDSC3001 - Assignment 3


## Question 1

In [ ]:
# sampling algorithm

# proof of correctness

## Question 2

### Part A

When an itemset $I$ has a size of $m$, there are $2^m -1$ possible subsets. When mining top-$k$ most frequent patterns, 

$$
2^m - 1 \leq k \\
2^m \leq k + 1 \\
m \leq \log_2(k + 1) \\
\therefore m = \lceil \log_2(k + 1) \rceil
$$

### Part B

#### b.1

#### b.2

#### b.3

#### b.4
Question B4:

Set $k=500$. Run your Misra–Gries Algorithm on the "trans.txt" dataset and report the values of $L$ and $minSup(A)$ when setting $C=500000, 750000, 1000000$. To compute $minSub(A)$, you can refer to the file "patterns_Apriori.txt" containing all the frequent patterns of support at least $21$. Each line of "patterns_Apriori.txt" is in the form $id_1,id_2,...,id_l:sup$, where $id_1,id_2,...,id_l$ denotes a pattern $\{id_1,id_2,...,id_l\}$ and $sup$ is the support of this pattern. (Hint: the file "patterns_Apriori.txt" contains enough information. If your algorithm returns some pattern that is not in the "patterns_Apriori.txt" file, probably your algorithm is not implemented correctly.)

In [ ]:
import numpy as np
import polars as pl
import math
from collections import Counter
from itertools import combinations

In [ ]:
class FrequenctPatterns:
    def __init__(self):
        self.transactions = []
        self.patterns = {}

    def load_data(self):
        with open("trans.txt") as f:
            for line in f:
                transaction = list(map(int, line.split()))
                self.transactions.append(transaction)

        with open("patterns_Apriori.txt") as f:
            for line in f:
                key, value = line.strip().split(":")
                key = tuple(map(int, key.split(",")))
                self.patterns[key] = int(value)

    def Misra_Gries(self, C, k=500): ...

In [ ]:
frequent_patterns = FrequenctPatterns()
frequent_patterns.load_data()

In [ ]:
for count in [500_000, 750_000, 1_000_000]:
    frequent_patterns.Misra_Gries(count)